In [1]:
import requests
import pandas as pd
import numpy as np

API_PATH = 'http://ergast.com/api/f1'

In [2]:
def getDriversByYear(year, fmt='json'):
    url = '{}/{}/drivers.{}'.format(API_PATH, year, fmt)
    r = requests.get(url)
    drivers_data = r.json()
    return drivers_data['MRData']['DriverTable']['Drivers']


def getDriversByYearDf(year):
    drivers_records = getDriversByYear(year)
    return pd.DataFrame.from_records(drivers_records)
    
drivers_2014 = getDriversByYearDf(2014)
drivers_2014

,code,dateOfBirth,driverId,familyName,givenName,nationality,permanentNumber,url
0,ALO,1981-07-29,alonso,Alonso,Fernando,Spanish,14,http://en.wikipedia.org/wiki/Fernando_Alonso
1,BIA,1989-08-03,jules_bianchi,Bianchi,Jules,French,17,http://en.wikipedia.org/wiki/Jules_Bianchi
2,BOT,1989-08-29,bottas,Bottas,Valtteri,Finnish,77,http://en.wikipedia.org/wiki/Valtteri_Bottas
3,BUT,1980-01-19,button,Button,Jenson,British,22,http://en.wikipedia.org/wiki/Jenson_Button
4,CHI,1991-04-21,chilton,Chilton,Max,British,4,http://en.wikipedia.org/wiki/Max_Chilton
5,ERI,1990-09-02,ericsson,Ericsson,Marcus,Swedish,9,http://en.wikipedia.org/wiki/Marcus_Ericsson
6,GRO,1986-04-17,grosjean,Grosjean,Romain,French,8,http://en.wikipedia.org/wiki/Romain_Grosjean
7,GUT,1991-08-05,gutierrez,Gutiérrez,Esteban,Mexican,21,http://en.wikipedia.org/wiki/Esteban_Guti%C3%A...
8,HAM,1985-01-07,hamilton,Hamilton,Lewis,British,44,http://en.wikipedia.org/wiki/Lewis_Hamilton
9,HUL,1987-08-19,hulkenberg,Hülkenberg,Nico,German,27,http://en.wikipedia.org/wiki/Nico_H%C3%BClkenberg


In [3]:
def getRacesByYear(year, fmt='json'):
    url = '{}/{}.{}'.format(API_PATH, year, fmt)
    r = requests.get(url)
    races_data = r.json()
    return races_data['MRData']['RaceTable']['Races']


def getRacesByYearDf(year):
    races_records = getRacesByYear(year)
    return pd.DataFrame({
        'round': [item['round'] for item in races_records],
        'raceName': [item['raceName'] for item in races_records],
        'circuitId': [item['Circuit']['circuitId'] for item in races_records]
    })

races_2014 = getRacesByYearDf(2014)
races_2014

,circuitId,raceName,round
0,albert_park,Australian Grand Prix,1
1,sepang,Malaysian Grand Prix,2
2,bahrain,Bahrain Grand Prix,3
3,shanghai,Chinese Grand Prix,4
4,catalunya,Spanish Grand Prix,5
5,monaco,Monaco Grand Prix,6
6,villeneuve,Canadian Grand Prix,7
7,red_bull_ring,Austrian Grand Prix,8
8,silverstone,British Grand Prix,9
9,hockenheimring,German Grand Prix,10


In [4]:
def getRaceResultsByYearRound(year, roundNum, fmt='json'):
    url = '{}/{}/{}/results.{}'.format(API_PATH, year, roundNum, fmt)
    r = requests.get(url)
    results_data = r.json()
    return results_data['MRData']['RaceTable']['Races'][0]


def getRaceResultsDf(year, roundNum):
    race_data = getRaceResultsByYearRound(year, roundNum)
    results = race_data['Results']
    
    carNum = pd.Series([item['number'] for item in results], dtype=np.int8)
    pos = pd.Series([item['position'] for item in results], dtype=np.int8)
    driverId = pd.Series([item['Driver']['driverId'] for item in results])
    constructorId = pd.Series([item['Constructor']['constructorId'] for item in results])
    grid = pd.Series([item['grid'] for item in results], dtype=np.int8)
    
    return pd.DataFrame({
        'carNum': carNum,
        'pos': pos,
        'driverId': driverId,
        'constructorId': constructorId,
        'grid': grid
    })


# race.results.data=rbind(race.results.data,data.frame(
#       carNum=as.integer(as.character(rrd[[i]]$number)),
#       pos=as.integer(as.character(rrd[[i]]$position)),
#       driverId=rrd[[i]]$Driver$driverId,
#       constructorId=rrd[[i]]$Constructor$constructorId,
#       grid=as.integer(as.character(rrd[[i]]$grid)),
#       laps=as.integer(as.character(rrd[[i]]$laps)),
#       status=rrd[[i]]$status,
#       #millitime=rrd[[i]]$Time$millis,
#       fastlapnum=hack1(rrd[[i]]),
#       fastlaptime=hack2(rrd[[i]]),
#       fastlaprank=hack3(rrd[[i]])
#     ))
# getRaceResultsByYearRound(2014, 2)
getRaceResultsDf(2014, 2)

,carNum,constructorId,driverId,grid,pos
0,44,mercedes,hamilton,1,1
1,6,mercedes,rosberg,3,2
2,1,red_bull,vettel,2,3
3,14,ferrari,alonso,4,4
4,27,force_india,hulkenberg,7,5
5,22,mclaren,button,10,6
6,19,williams,massa,13,7
7,77,williams,bottas,18,8
8,20,mclaren,kevin_magnussen,8,9
9,26,toro_rosso,kvyat,11,10
